In [4]:
import dspy
import os
from dotenv import load_dotenv
load_dotenv()
load_dotenv('.secrets')

/Users/vicente/Library/Caches/pypoetry/virtualenvs/gen-ai-demos-umKuaqgj-py3.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


True

In [5]:
class BoardGameResearchPlanner(dspy.Signature):
    """Takes the user query and plans deep research by figuring out subquestions that relate to the theme the user is interested in knowing more"""
    question = dspy.InputField(desc="The user's initial query when researching board games")
    subquestions: list[str] = dspy.OutputField(desc="A list of 5 or more subquestions that dive deeper in the theme the user is researching.")

class BoardGameResearcher(dspy.Signature):
    """Takes the original question and the related question and finds an answer to the subquestion that is related to the original question"""
    question = dspy.InputField(desc="The user's initial query about board games")
    subquestion: str = dspy.InputField(desc="The subquestion you are doing deep research on")
    context = dspy.OutputField(desc="The context learned by answering the subquestion while relevant to the initial question")

class BoardGameResearchSummarizer(dspy.Signature):
    """Condense the retrieved information provided in the context into a focused summary that answers the original question."""
    question = dspy.InputField(desc="The user's initial query about board games")
    context: list[str] = dspy.InputField(desc="Research context discovered about the board games you are about to recommend")
    summary = dspy.OutputField(desc="Condense retrieved information into a focused summary relevant to the provided context and user question. Provide specific details supporting the answer to the question.")


class BoardGameAssistant(dspy.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.plan_research = dspy.ChainOfThought(BoardGameResearchPlanner)
        self.research = dspy.ChainOfThought(BoardGameResearcher)
        self.summarize = dspy.ChainOfThought(BoardGameResearchSummarizer)

    def forward(self, question):
        subquestions = self.plan_research(question=question).subquestions
        research_context = []
        print(f"Question: {question} research tasks: {len(subquestions)}")
        for subquestion in subquestions:
            print(f" - Researching: {subquestion}")
            research = self.research(subquestion=subquestion, question=question)
            research_context = research_context + [research]

        final_research = self.summarize(context=list(map(lambda r: r.context, research_context)), question=question)

        return dspy.Prediction(answer=final_research.summary, research=research_context)

In [6]:
lm = dspy.LM('openai/gpt-4o', api_key=os.getenv("OPENAI_API_KEY"))
dspy.configure(lm=lm)

In [7]:
assistant = BoardGameAssistant()
recommendation = assistant(question="What is the best opening move in Catan?")

Question: What is the best opening move in Catan? research tasks: 7
 - Researching: What are the most important resources to prioritize in the early game of Catan?
 - Researching: How does the number of players affect the best opening move in Catan?
 - Researching: What are common strategies for initial settlement placement in Catan?
 - Researching: How does the board layout influence the choice of the best opening move?
 - Researching: What role does trading play in determining the best opening strategy in Catan?
 - Researching: How can a player adapt their opening move based on opponents' placements?
 - Researching: What are the risks and benefits of focusing on a single resource type in the opening move?


In [8]:
print(f"Answer: {recommendation.answer}")

Answer: The best opening move in Catan is highly situational and depends on the specific game setup, including the number of players and the board layout. Players should focus on securing key resources like brick and wood for road building, aim for resource diversity, and position themselves for advantageous trading opportunities. Adapting to the unique board configuration and opponents' placements is crucial for optimizing resource acquisition and expansion potential, making flexibility and strategic positioning essential components of a strong opening move.


In [9]:
print(f"Research: {recommendation.research}")

Research: [Prediction(
    reasoning='In the early game of Catan, the most important resources to prioritize are typically brick and wood. These resources are crucial because they are needed to build roads, which are essential for expanding your settlements and accessing more resource tiles. Additionally, having access to wheat and sheep is important for building settlements, which increase your resource production and victory points. Prioritizing these resources allows players to quickly expand their territory and gain a strategic advantage over opponents.',
    context='Understanding which resources to prioritize in the early game of Catan helps inform the best opening move. By focusing on securing access to brick and wood, a player can effectively plan their initial settlement placements to maximize road building potential. This strategy supports rapid expansion and increases the likelihood of controlling key areas of the board, which is a critical component of a strong opening move

In [10]:
lm.inspect_history(n=10)





[2025-02-26T13:45:01.764965]

System message:

Your input fields are:
1. `question` (str): The user's initial query when researching board games

Your output fields are:
1. `reasoning` (str)
2. `subquestions` (list[str]): A list of 5 or more subquestions that dive deeper in the theme the user is researching.

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## subquestions ## ]]
{subquestions}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Takes the user query and plans deep research by figuring out subquestions that relate to the theme the user is interested in knowing more


User message:

[[ ## question ## ]]
What is the best opening move in Catan?

Respond with the corresponding outpu

In [11]:
monopoly = BoardGameAssistant()
monoreco = assistant(question="What is the best opening move in Monopoly?")

Question: What is the best opening move in Monopoly? research tasks: 7
 - Researching: What are the probabilities of landing on each property in the first few turns of Monopoly?
 - Researching: Which properties are statistically the most valuable to own in Monopoly?
 - Researching: How do different house rules affect the strategy for the opening move in Monopoly?
 - Researching: What are common strategies used by experienced players for the opening move in Monopoly?
 - Researching: How does the initial roll of the dice influence the best opening move in Monopoly?
 - Researching: What is the impact of purchasing railroads early in the game?
 - Researching: How do player dynamics and negotiation strategies affect the opening moves in Monopoly?


In [12]:
print(monoreco.answer)

The best opening move in Monopoly is to prioritize acquiring properties with high landing probabilities, such as Oriental Avenue, Vermont Avenue, and the Orange properties, to maximize early game income. Securing railroads can also provide a consistent income stream. The strategy should be flexible, considering house rules and player dynamics, such as starting cash and negotiation styles, to effectively manage cash flow and property acquisition.


In [13]:
print(monoreco.research)

[Prediction(
    reasoning='In Monopoly, the probabilities of landing on each property in the first few turns are influenced by the roll of two six-sided dice, which determines movement. The most common roll is a 7, followed by 6 and 8, due to the number of combinations that can result in these sums. Therefore, properties that are 6, 7, or 8 spaces from the starting point (GO) are more likely to be landed on early in the game. These include properties like Oriental Avenue, Vermont Avenue, and Chance spaces. Understanding these probabilities helps in determining which properties are more strategic to purchase early on, as they are more likely to generate income from other players landing on them.',
    context='Knowing the probabilities of landing on certain properties in the first few turns of Monopoly can inform the best opening move. Since properties like Oriental Avenue and Vermont Avenue are more likely to be landed on early, purchasing them can be a strategic move. This increases 

## Self-Reflection Applied to RAG

Source: Asai, Akari, et al. “SELF-RAG: Learning to Retrieve, Generate, and Critique through Self-Reflection.” arXiv preprint arXiv:2310.11511 (2023). https://arxiv.org/abs/2310.11511

In [14]:
class BoardGameAdaptativeResearcher(dspy.Signature):
    """Takes the original question and the related question and finds an answer to the subquestion that is related to the original question."""
    question = dspy.InputField(desc="The user's initial query about board games")
    subquestion: str = dspy.InputField(desc="The subquestion you are doing deep research on")
    retrieve = dspy.OutputField(desc="Decide whether retrieval is necessary for this subquestion (Yes/No)")
    context = dspy.OutputField(desc="The context learned by answering the subquestion while relevant to the initial question")

class BoardGameCritic(dspy.Signature):
    """Assesses the quality and relevance of the retrieved context."""
    question = dspy.InputField(desc="The user's initial query about board games")
    subquestion: str = dspy.InputField(desc="The subquestion being researched")
    context = dspy.InputField(desc="The retrieved context for the subquestion")
    critique = dspy.OutputField(desc="Critique of whether the retrieved context is useful, partially useful, or irrelevant.")

class BoardGameAdaptativeAssistant(dspy.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.plan_research = dspy.ChainOfThought(BoardGameResearchPlanner)
        self.research = dspy.ChainOfThought(BoardGameAdaptativeResearcher)
        self.critic = dspy.ChainOfThought(BoardGameCritic)
        self.summarize = dspy.ChainOfThought(BoardGameResearchSummarizer)

    def forward(self, question):
        subquestions = self.plan_research(question=question).subquestions
        research_context = []
        print(f"Question: {question} research tasks: {len(subquestions)}")

        for subquestion in subquestions:
            print(f" - Evaluating retrieval need for: {subquestion}")
            research = self.research(subquestion=subquestion, question=question)

            if research.retrieve.lower() == "yes":
                print(f"   ✅Retrieval needed. Researching...")
                critic_response = self.critic(subquestion=subquestion, question=question, context=research.context)
                
                if "irrelevant" in critic_response.critique.lower():
                    print(f"   ❌Retrieved context deemed irrelevant. Skipping...")
                    continue
                else:
                    research_context.append(research.context)
            else:
                print(f"   ⏭️No retrieval needed. Skipping this subquestion.")

        final_research = self.summarize(context=research_context, question=question)

        return dspy.Prediction(answer=final_research.summary, research=research_context)

In [15]:
adaptative_assistant = BoardGameAdaptativeAssistant()
new_recommendation = adaptative_assistant(question="What is the best opening move in Catan?")

Question: What is the best opening move in Catan? research tasks: 7
 - Evaluating retrieval need for: What are the most important resources to prioritize in the early game of Catan?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: How does the number of players affect the best opening move in Catan?
   ✅Retrieval needed. Researching...
 - Evaluating retrieval need for: What are common strategies for initial settlement placement in Catan?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: How does the board layout influence the choice of the best opening move?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: What role does trading play in determining the best opening strategy in Catan?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: How can a player adapt their opening move based on opponents' placements?
   ⏭️No retrieval needed. Skippin

In [16]:
new_recommendation

Prediction(
    answer='The best opening move in Catan depends on the number of players. With fewer players, aim for a balanced resource strategy due to less competition. With more players, prioritize securing high-value resource spots early, focusing on intersections that offer a mix of brick, wood, and wheat to facilitate early development.',
    research=['The number of players in Catan affects the best opening move by altering the availability of resources and competition for key locations. With fewer players, there is more flexibility in choosing diverse resources and strategic positions, while with more players, securing high-value resource spots becomes more critical due to increased competition.']
)

In [17]:
strategy = adaptative_assistant(question="What is the best opening move in Catan? Which resource should I focus on? What is the best response if my opponent opens with that move?")

Question: What is the best opening move in Catan? Which resource should I focus on? What is the best response if my opponent opens with that move? research tasks: 9
 - Evaluating retrieval need for: What are the most common opening strategies in Catan and their advantages?
   ✅Retrieval needed. Researching...
 - Evaluating retrieval need for: How does the initial board setup influence the choice of opening move in Catan?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: Which resources are generally considered most valuable in the early game of Catan and why?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: How can I assess the probability of resource production based on the initial dice roll numbers?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: What are effective counter-strategies if an opponent secures a strong opening position?
   ✅Retrieval needed. Researching...
 -

In [18]:
strategy

Prediction(
    answer='The best opening move in Catan typically involves securing brick and wood to build roads and settlements quickly, or ore and wheat for upgrading to cities and developing cards. The choice depends on the board layout and resource availability. If an opponent opens with a strong move, counter by blocking their expansion, trading strategically, and prioritizing development cards to maintain competitiveness.',
    research=["The most common opening strategies in Catan involve securing key resources that enable early expansion or development. Players often prioritize brick and wood to build roads and settlements quickly, or ore and wheat to upgrade to cities and develop cards. The choice of strategy depends on the board layout and available resource numbers. Knowing these strategies helps in deciding the best opening move and how to counter an opponent's strategy effectively.", "Effective counter-strategies in Catan when an opponent secures a strong opening position 

In [19]:
adaptative_assistant = BoardGameAdaptativeAssistant()
result = adaptative_assistant(question="What's the best opening move in Catan? Explain in high detail 3 potential scenarios where it showcases the best performance.")


Question: What's the best opening move in Catan? Explain in high detail 3 potential scenarios where it showcases the best performance. research tasks: 8
 - Evaluating retrieval need for: What are the key resources in Catan, and how do they influence the opening move?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: How does the initial placement of settlements affect resource acquisition and strategy?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: What are the common strategies for the first settlement placement in Catan?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: How does the distribution of numbers on the board impact the effectiveness of an opening move?
   ⏭️No retrieval needed. Skipping this subquestion.
 - Evaluating retrieval need for: What role do ports play in determining the best opening move in Catan?
   ⏭️No retrieval needed. Skipping this subquestion.
 

In [20]:
from IPython.display import display, Markdown

display(Markdown(result['answer']))

The best opening move in Catan varies based on the board setup and opponents' strategies. In a scenario with limited space and high competition, prioritizing brick and wood is advantageous for rapid expansion. In contrast, focusing on wheat and ore is beneficial when high resource production is possible, allowing for quick upgrades to cities. A balanced approach, securing a mix of resources, is effective when the board is evenly distributed, providing flexibility to adapt to the game's progression. The optimal strategy involves assessing the board and opponents to determine the most advantageous resources to prioritize.